In [1]:
# Importando bibliotecas
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np
import MySQLdb
import matplotlib.pyplot as plt
import seaborn as sns
import time
import seaborn as sns
import matplotlib.pyplot as plt
start_time = time.time()
%matplotlib inline



array([1.e+00, 1.e-01, 1.e-02, 1.e-03, 1.e-04, 1.e-05, 1.e-06, 1.e-07,
       1.e-08, 1.e-09])

In [2]:
# Lendo o banco de dados
con = MySQLdb.connect(   host = "localhost",
                        user = "XXX",
                        passwd = "XXX",
                        db = "Dataset")

df = pd.read_sql('SELECT * FROM Dataset where area = 1', con)
df.fillna({'Objetivo_da_Operacao':0, 'Aplicacao_de_Recursos':0, 'Agricultura_Familiar':0, 'Tipo_de_Assistencia_Tecnica':0, 'Amparo_da_Operacao':0, 'Grupo_do_Cliente':0, 'Total_do_FNE_Risco':0, 'Tipo_de_Tratamento':0, 'Forma_de_Credito':0, 'Incidencia_de_IOF':0, 'Custo_Efetivo_Total':0, 'Atividade':0, 'Finalidade_do_Financiamento':0, 'Categoria_do_Produtor':0, 'Tipo_de_Recurso':0, 'Tipo_Lei':0, 'Operacao_Consignacao':0}, inplace = True)

In [4]:
def remove_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
    return df_out

In [3]:
dataset = df[['Objetivo_da_Operacao', 'Tempo_Em_Anos', 'Pendencia_de_Cobertura_do_PROAGRO', 'Tipo_de_Assistencia_Tecnica', 'Tipo_de_Renegociacao', 'Faixa_valores', 
        'Tipo_Lei', 'Tem_Bonus', 'Tem_Debito_Evento_Anual', 'Tem_Debito_Evento_Mensal', 'E_Renegociada', 'Tem_Prejuizo']]


In [5]:
dataset.sample(10)

,Objetivo_da_Operacao,Tempo_Em_Anos,Pendencia_de_Cobertura_do_PROAGRO,Tipo_de_Assistencia_Tecnica,Tipo_de_Renegociacao,Faixa_valores,Tipo_Lei,Tem_Bonus,Tem_Debito_Evento_Anual,Tem_Debito_Evento_Mensal,E_Renegociada,Tem_Prejuizo
1661,1.0,1.0,0.0,1.0,0.0,4.0,0.0,1.0,0.0,1.0,0.0,0.0
720,1.0,2.0,1.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0
981,1.0,1.0,0.0,1.0,0.0,4.0,0.0,1.0,0.0,1.0,0.0,0.0
1261,1.0,1.0,0.0,1.0,0.0,4.0,0.0,1.0,0.0,1.0,0.0,0.0
719,1.0,1.0,1.0,0.0,0.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0
699,1.0,2.0,1.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0
1842,1.0,2.0,0.0,1.0,2.0,4.0,0.0,1.0,0.0,0.0,1.0,1.0
1572,1.0,2.0,0.0,1.0,0.0,4.0,0.0,1.0,0.0,1.0,0.0,0.0
39,1.0,1.0,1.0,1.0,0.0,4.0,0.0,1.0,0.0,1.0,0.0,0.0
1383,1.0,2.0,0.0,1.0,0.0,4.0,0.0,1.0,0.0,1.0,0.0,0.0


# Seleção de variáveis e execução de modelos de Machine Learning

In [16]:
#operações com chance de prejuizo
X = df[['Objetivo_da_Operacao', 'Tempo_Em_Anos', 'Pendencia_de_Cobertura_do_PROAGRO', 'Tipo_de_Assistencia_Tecnica', 'Tipo_de_Renegociacao', 'Faixa_valores', 
        'Tipo_Lei', 'Tem_Bonus', 'Tem_Debito_Evento_Anual', 'Tem_Debito_Evento_Mensal', 'E_Renegociada']]

y = df['Tem_Prejuizo']

# Definindo as métricas de avaliação
scoring = ['accuracy', 'precision','recall', 'f1', 'roc_auc']

# Variaveis para armazenar o melhor classificador de cada modelo
knn_best_estimator = ""
knn_best_score = 0
nb_best_estimator = ""
nb_best_score = 0
nb_distribution = ""
dt_best_estimator = ""
dt_best_score = 0
rf_best_estimator = ""
rf_best_score = 0
mlp_best_estimator = ""
mlp_best_score = 0
svc_best_estimator = ""
svc_best_score = 0
svc_kernel = ""

## KNN

In [17]:
# Instanciando o classificador
knn = KNeighborsClassifier(n_jobs=-1)

# Definindo alguns parametros para teste
params = {
    'n_neighbors': [2,3,5,10, 15,20],
    'metric': ['euclidean','manhattan','chebyshev']
}

In [18]:
# Instanciando o gridsearch e executando o modelo

for score_item in scoring:
    print('Métrica de avaliacao: ', score_item)
    grid_search = GridSearchCV(estimator=knn,
                           param_grid=params,
                           cv = 5,
                           n_jobs=-1, verbose=1, scoring=score_item)
    #%%time
    grid_search.fit(X, y)
    best = grid_search.best_estimator_
    print('Melhor classificador: ',best)
    print('Score: ',grid_search.best_score_)
    print()
    if (knn_best_score < grid_search.best_score_):
        knn_best_estimator = best
        knn_best_score = grid_search.best_score_
    

Métrica de avaliacao:  accuracy
Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:    3.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Melhor classificador:  KNeighborsClassifier(metric='manhattan', n_jobs=-1, n_neighbors=2)
Score:  0.8292685288280002

Métrica de avaliacao:  precision
Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:    1.6s finished


Melhor classificador:  KNeighborsClassifier(metric='chebyshev', n_jobs=-1, n_neighbors=3)
Score:  0.4884318601186071

Métrica de avaliacao:  recall
Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:    1.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Melhor classificador:  KNeighborsClassifier(metric='chebyshev', n_jobs=-1, n_neighbors=3)
Score:  0.4688524590163934

Métrica de avaliacao:  f1
Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:    1.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Melhor classificador:  KNeighborsClassifier(metric='chebyshev', n_jobs=-1, n_neighbors=3)
Score:  0.37820822153523687

Métrica de avaliacao:  roc_auc
Fitting 5 folds for each of 18 candidates, totalling 90 fits
Melhor classificador:  KNeighborsClassifier(metric='manhattan', n_jobs=-1, n_neighbors=10)
Score:  0.6745439296026982



[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:    1.1s finished


## Naive Bayes

In [19]:
nb = GaussianNB()
params = {'var_smoothing': np.logspace(0,-9, num=10)}

for score_item in scoring:
    print('Métrica de avaliacao: ', score_item)
    grid_search = GridSearchCV(estimator=nb,
                            param_grid=params,
                            cv = 5,
                            n_jobs=-1, verbose=1, scoring=score_item)
    #%%time
    grid_search.fit(X, y)
    best = grid_search.best_estimator_
    print('Melhor classificador: ',best)
    print('Score: ',grid_search.best_score_)
    print()
    if (nb_best_score < grid_search.best_score_):
        nb_best_estimator = best
        nb_best_score = grid_search.best_score_
        nb_distribution = 'Gaussian'

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Métrica de avaliacao:  accuracy
Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Melhor classificador:  GaussianNB(var_smoothing=1.0)
Score:  0.8657568862855207

Métrica de avaliacao:  precision
Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


...

[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.2s finished


Melhor classificador:  GaussianNB(var_smoothing=1e-05)
Score:  0.6032786885245902

Métrica de avaliacao:  f1
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Melhor classificador:  GaussianNB(var_smoothing=1e-05)
Score:  0.45515752366465206

Métrica de avaliacao:  roc_auc
Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Melhor classificador:  GaussianNB(var_smoothing=0.01)
Score:  0.7443198471376744



[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.2s finished


In [20]:
nb = MultinomialNB()
params = {
    'alpha': [0.1,0.2,0.3,0.5,0.7,1.0]
}

for score_item in scoring:
    print('Métrica de avaliacao: ', score_item)
    grid_search = GridSearchCV(estimator=nb,
                            param_grid=params,
                            cv = 5,
                            n_jobs=-1, verbose=1, scoring=score_item)
    #%%time
    grid_search.fit(X, y)
    best = grid_search.best_estimator_
    print('Melhor classificador: ',best)
    print('Score: ',grid_search.best_score_)
    print()
    if (nb_best_score < grid_search.best_score_):
        nb_best_estimator = best
        nb_best_score = grid_search.best_score_
        nb_distribution = 'Multinomial'

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  23 out of  30 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Métrica de avaliacao:  accuracy
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Melhor classificador:  MultinomialNB()
Score:  0.7892491649319844

Métrica de avaliacao:  precision
Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  23 out of  30 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Melhor classificador:  MultinomialNB()
Score:  0.3467374950416072

Métrica de avaliacao:  recall
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Melhor classificador:  MultinomialNB(alpha=0.1)
Score:  0.4655737704918034

Métrica de avaliacao:  f1
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Melhor classificador:  MultinomialNB()
Score:  0.3637544502078056

Métrica de avaliacao:  roc_auc
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Melhor classificador:  MultinomialNB()
Score:  0.7187521452327609



[Parallel(n_jobs=-1)]: Done  23 out of  30 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    0.1s finished


In [21]:
nb = BernoulliNB()
params = {
    'alpha': [0.1,0.2,0.3,0.5,0.7,1.0]
}
for score_item in scoring:
    print('Métrica de avaliacao: ', score_item)
    grid_search = GridSearchCV(estimator=nb,
                            param_grid=params,
                            cv = 5,
                            n_jobs=-1, verbose=1, scoring=score_item)
    #%%time
    grid_search.fit(X, y)
    best = grid_search.best_estimator_
    print('Melhor classificador: ',best)
    print('Score: ',grid_search.best_score_)
    print()
    if (nb_best_score < grid_search.best_score_):
        nb_best_estimator = best
        nb_best_score = grid_search.best_score_
        nb_distribution = 'Bernoulli'

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Métrica de avaliacao:  accuracy
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Melhor classificador:  BernoulliNB(alpha=0.1)
Score:  0.7844294912136323

Métrica de avaliacao:  precision
Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Done  23 out of  30 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  23 out of  30 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Melhor classificador:  BernoulliNB(alpha=0.1)
Score:  0.4395043028624233

Métrica de avaliacao:  recall
Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Done  23 out of  30 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Melhor classificador:  BernoulliNB(alpha=0.1)
Score:  0.5836065573770491

Métrica de avaliacao:  f1
Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Done  23 out of  30 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Melhor classificador:  BernoulliNB(alpha=0.1)
Score:  0.46447163527341384

Métrica de avaliacao:  roc_auc
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Melhor classificador:  BernoulliNB(alpha=0.1)
Score:  0.7714813894904188



[Parallel(n_jobs=-1)]: Done  23 out of  30 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    0.1s finished


## Decision Tree

In [22]:
dt = DecisionTreeClassifier(random_state=42)
params = {
    'criterion': ['gini','entropy'],
    'max_depth': [2,3,5,10,20],
    'min_samples_leaf': [5,10,20,50,100,200]
}
for score_item in scoring:
    print('Métrica de avaliacao: ', score_item)
    grid_search = GridSearchCV(estimator=dt,
                            param_grid=params,
                            cv = 5,
                            n_jobs=-1, verbose=1, scoring=score_item)
    #%%time
    grid_search.fit(X, y)
    best = grid_search.best_estimator_
    print('Melhor classificador: ',best)
    print('Score: ',grid_search.best_score_)
    print()
    if (dt_best_score < grid_search.best_score_):
        dt_best_estimator = best
        dt_best_score = grid_search.best_score_

Métrica de avaliacao:  accuracy
Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 250 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:    1.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Melhor classificador:  DecisionTreeClassifier(max_depth=2, min_samples_leaf=100, random_state=42)
Score:  0.884665730745026

Métrica de avaliacao:  precision
Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Done 235 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:    1.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Melhor classificador:  DecisionTreeClassifier(max_depth=2, min_samples_leaf=100, random_state=42)
Score:  0.6018078048923996

Métrica de avaliacao:  recall
Fitting 5 folds for each of 60 candidates, totalling 300 fits


...

[Parallel(n_jobs=-1)]: Done 250 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:    1.0s finished


Melhor classificador:  DecisionTreeClassifier(max_depth=2, min_samples_leaf=100, random_state=42)
Score:  0.5406673769959096

Métrica de avaliacao:  roc_auc
Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 250 tasks      | elapsed:    0.9s


Melhor classificador:  DecisionTreeClassifier(max_depth=2, min_samples_leaf=5, random_state=42)
Score:  0.7573554618658342



[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:    1.1s finished


## Random Forest

In [23]:
start_time = time.time()
rf = RandomForestClassifier(random_state=42)
params = {
    'criterion': ['gini','entropy'],
    'max_depth': [2,3,5,10,20,40],
    'min_samples_leaf': [5,10,20,50,75,100,200],
    'n_estimators': [10,25,30,50,75,100,200]
}
for score_item in scoring:
    print('Métrica de avaliacao: ', score_item)
    grid_search = GridSearchCV(estimator=rf,
                            param_grid=params,
                            cv = 4,
                            n_jobs=-1, verbose=1, scoring=score_item)
    #%%time
    grid_search.fit(X, y)
    best = grid_search.best_estimator_
    print('Melhor classificador: ',best)
    print('Score: ',grid_search.best_score_)
    print()
    if (rf_best_score < grid_search.best_score_):
        rf_best_estimator = best
        rf_best_score = grid_search.best_score_
end_time = time.time()
print('Tempo total: ', end_time-start_time)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Métrica de avaliacao:  accuracy
Fitting 4 folds for each of 588 candidates, totalling 2352 fits


[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 376 tasks      | elapsed:   19.8s
[Parallel(n_jobs=-1)]: Done 876 tasks      | elapsed:   52.1s
[Parallel(n_jobs=-1)]: Done 1576 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 2352 out of 2352 | elapsed:  2.5min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Melhor classificador:  RandomForestClassifier(criterion='entropy', max_depth=5, min_samples_leaf=10,
                       n_estimators=25, random_state=42)
Score:  0.9018485915492958

Métrica de avaliacao:  precision
Fitting 4 folds for each of 588 candidates, totalling 2352 fits


[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done 376 tasks      | elapsed:   27.0s
[Parallel(n_jobs=-1)]: Done 876 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 1576 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 2352 out of 2352 | elapsed:  2.8min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done 376 tasks      | elapsed:   26.2s
[Parallel(n_jobs=-1)]: Done 876 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 1576 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 2352 out of 2352 | elapsed:  2.7min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Melhor classificador:  RandomForestClassifier(criterion='entropy', max_depth=3, min_samples_leaf=20,
                       n_estimators=75, random_state=42)
Score:  0.4627375453006636

Métrica de avaliacao:  roc_auc
Fitting 4 folds for each of 588 candidates, totalling 2352 fits


[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done 376 tasks      | elapsed:   25.5s
[Parallel(n_jobs=-1)]: Done 876 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 1576 tasks      | elapsed:  1.9min


Melhor classificador:  RandomForestClassifier(criterion='entropy', max_depth=5, min_samples_leaf=10,
                       n_estimators=50, random_state=42)
Score:  0.834844404520775

Tempo total:  809.0458266735077


[Parallel(n_jobs=-1)]: Done 2352 out of 2352 | elapsed:  2.8min finished


In [24]:
#Classificador 1
rf = RandomForestClassifier(random_state=42)
params = {
    'criterion': ['entropy'],
    'max_depth': [20],
    'min_samples_leaf': [5],
    'n_estimators': [50]
}
for score_item in scoring:
    print('Métrica de avaliacao: ', score_item)
    grid_search = GridSearchCV(estimator=rf,
                            param_grid=params,
                            cv = 5,
                            n_jobs=-1, verbose=1, scoring=score_item)
    #%%time
    grid_search.fit(X, y)
    best = grid_search.best_estimator_
    print('Melhor classificador: ',best)
    print('Score: ',grid_search.best_score_)
    print()
    if (rf_best_score < grid_search.best_score_):
        rf_best_estimator = best
        rf_best_score = grid_search.best_score_

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Métrica de avaliacao:  accuracy
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Melhor classificador:  RandomForestClassifier(criterion='entropy', max_depth=20, min_samples_leaf=5,
                       n_estimators=50, random_state=42)
Score:  0.8490593987510288

Métrica de avaliacao:  precision
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


...

[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Melhor classificador:  RandomForestClassifier(criterion='entropy', max_depth=20, min_samples_leaf=5,
                       n_estimators=50, random_state=42)
Score:  0.37377049180327865

Métrica de avaliacao:  f1
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Melhor classificador:  RandomForestClassifier(criterion='entropy', max_depth=20, min_samples_leaf=5,
                       n_estimators=50, random_state=42)
Score:  0.3454539568880273

Métrica de avaliacao:  roc_auc
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Melhor classificador:  RandomForestClassifier(criterion='entropy', max_depth=20, min_samples_leaf=5,
                       n_estimators=50, random_state=42)
Score:  0.7163721411630386



[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.3s finished


In [25]:
#Classificador 2
rf = RandomForestClassifier(random_state=42)
params = {
    'criterion': ['entropy'],
    'max_depth': [10],
    'min_samples_leaf': [20],
    'n_estimators': [10]
}
for score_item in scoring:
    print('Métrica de avaliacao: ', score_item)
    grid_search = GridSearchCV(estimator=rf,
                            param_grid=params,
                            cv = 5,
                            n_jobs=-1, verbose=1, scoring=score_item)
    #%%time
    grid_search.fit(X, y)
    best = grid_search.best_estimator_
    print('Melhor classificador: ',best)
    print('Score: ',grid_search.best_score_)
    print()
    if (rf_best_score < grid_search.best_score_):
        rf_best_estimator = best
        rf_best_score = grid_search.best_score_

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Métrica de avaliacao:  accuracy
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Melhor classificador:  RandomForestClassifier(criterion='entropy', max_depth=10, min_samples_leaf=20,
                       n_estimators=10, random_state=42)
Score:  0.8547814300237209

Métrica de avaliacao:  precision
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Melhor classificador:  RandomForestClassifier(criterion='entropy', max_depth=10, min_samples_leaf=20,
                       n_estimators=10, random_state=42)
Score:  0.47287072598559243

Métrica de avaliacao:  recall
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Melhor classificador:  RandomForestClassifier(criterion='entropy', max_depth=10, min_samples_leaf=20,
                       n_estimators=10, random_state=42)
Score:  0.42950819672131135

Métrica de avaliacao:  f1
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Melhor classificador:  RandomForestClassifier(criterion='entropy', max_depth=10, min_samples_leaf=20,
                       n_estimators=10, random_state=42)
Score:  0.40426295618304503

Métrica de avaliacao:  roc_auc
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Melhor classificador:  RandomForestClassifier(criterion='entropy', max_depth=10, min_samples_leaf=20,
                       n_estimators=10, random_state=42)
Score:  0.7038884692825962



[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.1s finished


In [26]:
#Classificador 3
rf = RandomForestClassifier(random_state=42)
params = {
    'criterion': ['entropy'],
    'max_depth': [2],
    'min_samples_leaf': [200],
    'n_estimators': [50]
}
for score_item in scoring:
    print('Métrica de avaliacao: ', score_item)
    grid_search = GridSearchCV(estimator=rf,
                            param_grid=params,
                            cv = 5,
                            n_jobs=-1, verbose=1, scoring=score_item)
    #%%time
    grid_search.fit(X, y)
    best = grid_search.best_estimator_
    print('Melhor classificador: ',best)
    print('Score: ',grid_search.best_score_)
    print()
    if (rf_best_score < grid_search.best_score_):
        rf_best_estimator = best
        rf_best_score = grid_search.best_score_

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Métrica de avaliacao:  accuracy
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Melhor classificador:  RandomForestClassifier(criterion='entropy', max_depth=2, min_samples_leaf=200,
                       n_estimators=50, random_state=42)
Score:  0.8657568862855207

Métrica de avaliacao:  precision
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


...

[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Melhor classificador:  RandomForestClassifier(criterion='entropy', max_depth=2, min_samples_leaf=200,
                       n_estimators=50, random_state=42)
Score:  0.0

Métrica de avaliacao:  f1
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Melhor classificador:  RandomForestClassifier(criterion='entropy', max_depth=2, min_samples_leaf=200,
                       n_estimators=50, random_state=42)
Score:  0.0

Métrica de avaliacao:  roc_auc
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Melhor classificador:  RandomForestClassifier(criterion='entropy', max_depth=2, min_samples_leaf=200,
                       n_estimators=50, random_state=42)
Score:  0.7054657936879496



[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.3s finished


In [27]:
#Classificador 4
rf = RandomForestClassifier(random_state=42)
params = {
    'criterion': ['entropy'],
    'max_depth': [20],
    'min_samples_leaf': [5],
    'n_estimators': [200]
}
for score_item in scoring:
    print('Métrica de avaliacao: ', score_item)
    grid_search = GridSearchCV(estimator=rf,
                            param_grid=params,
                            cv = 5,
                            n_jobs=-1, verbose=1, scoring=score_item)
    #%%time
    grid_search.fit(X, y)
    best = grid_search.best_estimator_
    print('Melhor classificador: ',best)
    print('Score: ',grid_search.best_score_)
    print()
    if (rf_best_score < grid_search.best_score_):
        rf_best_estimator = best
        rf_best_score = grid_search.best_score_

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Métrica de avaliacao:  accuracy
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.3s finished


Melhor classificador:  RandomForestClassifier(criterion='entropy', max_depth=20, min_samples_leaf=5,
                       n_estimators=200, random_state=42)
Score:  0.8508215132884736

Métrica de avaliacao:  precision
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.2s finished


...

Melhor classificador:  RandomForestClassifier(criterion='entropy', max_depth=20, min_samples_leaf=5,
                       n_estimators=200, random_state=42)
Score:  0.3868852459016393

Métrica de avaliacao:  f1
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.2s finished


Melhor classificador:  RandomForestClassifier(criterion='entropy', max_depth=20, min_samples_leaf=5,
                       n_estimators=200, random_state=42)
Score:  0.3604332308198677

Métrica de avaliacao:  roc_auc
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.2s finished


Melhor classificador:  RandomForestClassifier(criterion='entropy', max_depth=20, min_samples_leaf=5,
                       n_estimators=200, random_state=42)
Score:  0.7164786378753931



## MLP

In [28]:
mlp = MLPClassifier(max_iter=100)
params = {
    'hidden_layer_sizes': [(10,30,10),(20,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}
for score_item in scoring:
    print('Métrica de avaliacao: ', score_item)
    grid_search = GridSearchCV(estimator=mlp,
                            param_grid=params,
                            cv = 5,
                            n_jobs=-1, verbose=1, scoring=score_item)
    #%%time
    grid_search.fit(X, y)
    best = grid_search.best_estimator_
    print('Melhor classificador: ',best)
    print('Score: ',grid_search.best_score_)
    print()
    if (mlp_best_score < grid_search.best_score_):
        mlp_best_estimator = best
        mlp_best_score = grid_search.best_score_

Métrica de avaliacao:  accuracy
Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   23.7s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:  1.1min finished
/home/marcelo/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Melhor classificador:  MLPClassifier(activation='tanh', hidden_layer_sizes=(10, 30, 10),
              learning_rate='adaptive', max_iter=100, solver='sgd')
Score:  0.877190298688096

Métrica de avaliacao:  precision
Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   28.5s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:  1.1min finished
/home/marcelo/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Melhor classificador:  MLPClassifier(activation='tanh', hidden_layer_sizes=(10, 30, 10),
              learning_rate='adaptive', max_iter=100)
Score:  0.5278754911656999

Métrica de avaliacao:  recall
Fitting 5 folds for each of 32 candidates, totalling 160 fits


...

[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   27.8s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:  1.4min finished
/home/marcelo/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Melhor classificador:  MLPClassifier(hidden_layer_sizes=(10, 30, 10), max_iter=100)
Score:  0.4150223177252654

Métrica de avaliacao:  roc_auc
Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   21.1s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:  1.4min finished


Melhor classificador:  MLPClassifier(activation='tanh', alpha=0.05, hidden_layer_sizes=(20,),
              learning_rate='adaptive', max_iter=100)
Score:  0.8043137573763716



/home/marcelo/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


## SVM

In [29]:
#kernel rbf
svc_kernel = 'rbf'
svc = SVC(kernel='rbf')
params = {'C': [0.1, 1, 10], 
          'gamma': [1,0.1, 0.01]}
for score_item in scoring:
    print('Métrica de avaliacao: ', score_item)
    grid_search = GridSearchCV(estimator=svc,
                            param_grid=params,
                            cv = 5,
                            n_jobs=-1, verbose=1, scoring=score_item)
    #%%time
    grid_search.fit(X, y)
    best = grid_search.best_estimator_
    print('Melhor classificador: ',best)
    print('Score: ',grid_search.best_score_)
    print()
    if (svc_best_score < grid_search.best_score_):
        svc_best_estimator = best
        svc_best_score = grid_search.best_score_
        

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Métrica de avaliacao:  accuracy
Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Done  38 out of  45 | elapsed:    1.0s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:    1.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Melhor classificador:  SVC(C=0.1, gamma=0.01)
Score:  0.8547659389069082

Métrica de avaliacao:  precision
Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Done  38 out of  45 | elapsed:    0.8s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:    1.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


...

[Parallel(n_jobs=-1)]: Done  38 out of  45 | elapsed:    0.8s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:    1.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Melhor classificador:  SVC(C=10, gamma=1)
Score:  0.38032786885245906

Métrica de avaliacao:  f1
Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Done  38 out of  45 | elapsed:    0.8s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:    1.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Melhor classificador:  SVC(C=1, gamma=0.1)
Score:  0.332980977201007

Métrica de avaliacao:  roc_auc
Fitting 5 folds for each of 9 candidates, totalling 45 fits
Melhor classificador:  SVC(C=10, gamma=1)
Score:  0.7346667284959538



[Parallel(n_jobs=-1)]: Done  38 out of  45 | elapsed:    0.9s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:    1.0s finished


In [30]:
#kernel linear
svc_kernel = 'linear'
svc = SVC(kernel='linear')
params = {'C': [0.1,1,10]}
for score_item in scoring:
    print('Métrica de avaliacao: ', score_item)
    grid_search = GridSearchCV(estimator=svc,
                            param_grid=params,
                            cv = 5,
                            n_jobs=-1, verbose=1, scoring=score_item)
    #%%time
    grid_search.fit(X, y)
    best = grid_search.best_estimator_
    print('Melhor classificador: ',best)
    print('Score: ',grid_search.best_score_)
    print()
    if (svc_best_score < grid_search.best_score_):
        svc_best_estimator = best
        svc_best_score = grid_search.best_score_

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Métrica de avaliacao:  accuracy
Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    3.1s finished


Melhor classificador:  SVC(C=1, kernel='linear')
Score:  0.8402614125962143

Métrica de avaliacao:  precision
Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    3.2s finished


...

Melhor classificador:  SVC(C=0.1, kernel='linear')
Score:  0.5278688524590164

Métrica de avaliacao:  f1
Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    3.1s finished


Melhor classificador:  SVC(C=10, kernel='linear')
Score:  0.46610406363242696

Métrica de avaliacao:  roc_auc
Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    3.4s finished


Melhor classificador:  SVC(C=10, kernel='linear')
Score:  0.7625739066432816



In [31]:
#kernel polinomial
svc_kernel = 'polinomial'
svc = SVC(kernel='poly')
params = {'C': [0.1, 1, 10], 
          'degree': [3,4]}
for score_item in scoring:
    print('Métrica de avaliacao: ', score_item)
    grid_search = GridSearchCV(estimator=svc,
                            param_grid=params,
                            cv = 5,
                            n_jobs=-1, verbose=1, scoring=score_item)
    #%%time
    grid_search.fit(X, y)
    best = grid_search.best_estimator_
    print('Melhor classificador: ',best)
    print('Score: ',grid_search.best_score_)
    print()
    if (svc_best_score < grid_search.best_score_):
        svc_best_estimator = best
        svc_best_score = grid_search.best_score_
        

Métrica de avaliacao:  accuracy
Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  23 out of  30 | elapsed:    1.2s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    2.3s finished


Melhor classificador:  SVC(C=10, kernel='poly')
Score:  0.8578651304642493

Métrica de avaliacao:  precision
Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  23 out of  30 | elapsed:    1.2s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    2.2s finished


Melhor classificador:  SVC(C=10, degree=4, kernel='poly')
Score:  0.6298373311276537

Métrica de avaliacao:  recall
Fitting 5 folds for each of 6 candidates, totalling 30 fits


...

[Parallel(n_jobs=-1)]: Done  23 out of  30 | elapsed:    1.2s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    2.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Melhor classificador:  SVC(C=1, kernel='poly')
Score:  0.3600448098708245

Métrica de avaliacao:  roc_auc
Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Done  23 out of  30 | elapsed:    1.3s remaining:    0.4s


Melhor classificador:  SVC(C=1, kernel='poly')
Score:  0.7562343296106597



[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    2.2s finished


In [32]:
print(knn_best_estimator)
print(knn_best_score)
print(nb_best_estimator)
print(nb_best_score)
print(nb_distribution)
print(dt_best_estimator)
print(dt_best_score)
print(rf_best_estimator)
print(rf_best_score)
print(mlp_best_estimator)
print(mlp_best_score)

KNeighborsClassifier(metric='manhattan', n_jobs=-1, n_neighbors=2)
0.8292685288280002
GaussianNB(var_smoothing=1.0)
0.8657568862855207
Gaussian
DecisionTreeClassifier(max_depth=2, min_samples_leaf=100, random_state=42)
0.884665730745026
RandomForestClassifier(criterion='entropy', max_depth=5, min_samples_leaf=10,
                       n_estimators=25, random_state=42)
0.9018485915492958
MLPClassifier(activation='tanh', hidden_layer_sizes=(10, 30, 10),
              learning_rate='adaptive', max_iter=100, solver='sgd')
0.877190298688096
